In [29]:
import chardet
import csv
import gensim
import logging
import nltk
import os
import string

%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from itertools import cycle
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from gensim.models import Doc2Vec
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

# Upload Essays 

In [2]:
essay_path = "./APWA_text/Essays/"

In [3]:
files = os.listdir(essay_path)
# files.remove(".DS_Store")
essays = {}
compound_mapping = [ ("broken home", "brokenhome"), ("fit in", "fitin"), ("home boys", "homeboys"),
("crime partners", "crimepartners"), ("road dogs", "roaddogs"), ("step father", "stepfather"),
("old lady", "oldlady"), ("strip search", "stripsearch"), ("pecking order", "peckingorder"),
("solid crime", "solidcrime"), ("dirty crime", "dirtycrime"), ("skin crime", "skincrime"),
("solid prisoner", "solidprisoner"), ("dick sucker", "dicksucker"), ("cock sucker", "cocksucker"),
("shot caller", "shotcaller"), ("butt pirate", "buttpirate"), ("falsely accuse", "falselyaccuse"),
("born again", "bornagain"), ("good guy", "goodguy"), ("habeas corpus", "habeascorpus"),
("time barred", "timebarred"), ("successive petitions", "successivepetitions"), ("hunger strike", "hungerstrike"),
("1983 lawsuits", "1983lawsuits"), ("civil rights complaints", "civilrightscomplaints"), ("tax payers", "taxpayers"),
("private prisons", "privateprisons"), ("prison-industrial complex", "prison-industrialcomplex"), ("make money off of us", "makemoneyoffofus"),
("higher education", "highereducation"), ("correspondence courses", "correspondencecourses"), ("self help", "selfhelp"),
("mental health", "mentalhealth"), ("psych meds", "psychmeds"), ("kehea meds", "keheameds"), ("deliberate indifference", "deliberateindifference") ]
for file in files:
    # attempt to confidently guess encoding; otherwise, default to ISO-8859-1
    encoding = "ISO-8859-1"
    guess = chardet.detect(open(essay_path + file, "rb").read())
    if (guess["confidence"] >= 0.95):
        encoding = guess["encoding"]
    
    with open(essay_path + file, "r", encoding=encoding) as f:
        essays[file] = f.read()
        for k, v in compound_mapping:
            essays[file] = essays[file].replace(k, v) # replaces spaced words with their compounded versions

# Tokenize + Preprocess

In [4]:
tokenized_essays = {label: gensim.utils.simple_preprocess(corpus, deacc=True, min_len=2, max_len=20) for (label, corpus) in essays.items()}

In [5]:
english_stopwords = nltk.corpus.stopwords.words('english')

custom_stopwords = [
        "prison",
        "jail",
        "prisoner",
        "also",
        "said",
        "would",
        "could",
        "should",
        "first",
        "like",
        "get",
        "going",
        "thing",
        "something",
        "use",
        "get",
        "go",
        "one",
        "mr",
        "many",
        "much",
        "see",
        "take",
        "another",
        "aroud",
        "away",
        "back",
        "even",
        "every",
        "guy",
        "know",
        "let",
        "make",
        "look",
        "person",
        "place",
        "day",
        "year",
        "well",
        "good",
        "bad",
        "with",
        "without",
        "may",
        "new",
        "two",
        "want",
        "people",
        "within",
        "upon",
        "come",
        "tilocblob",
        "way",
        "around",
        "high",
        "inside",
        "long",
        "men",
        "must",
        "need",
        "never",
        "old",
        "other",
        "others",
        "really",
        "say",
        "seem",
        "still",
        "try",
        "become",
        "allow",
        "give",
        "month",
        "result",
        "always",
        "ask",
        "begin",
        "end",
        "hour",
        "man",
        "woman",
        "put",
        "someone",
        "start",
        "next",
        "act",
        "create",
        "yet",
        "time",
        "case",
        "cell",
        "work",
        "call",
        "world",
        "tell",
        "week",
        "told",
        "lot",
        "change",
        "self",
        "since",
        "life",
        "u"
    ]


In [6]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": nltk.corpus.wordnet.ADJ,
                "N": nltk.corpus.wordnet.NOUN,
                "V": nltk.corpus.wordnet.VERB,
                "R": nltk.corpus.wordnet.ADV}

    return tag_dict.get(tag, nltk.corpus.wordnet.NOUN)

lemmatizer = nltk.stem.WordNetLemmatizer()
tokenized_essays = {label: [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in token_lst if (w not in string.punctuation and w not in english_stopwords and w not in custom_stopwords)] for (label, token_lst) in tokenized_essays.items()}

In [7]:
# lst = [homies, homeboys, brothers, partners, crimies, road dogs, dogs, commitment offense, controlling offense]
# print([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in lst])

SyntaxError: invalid syntax (<ipython-input-7-c1ae1aa2b6c3>, line 1)

In [8]:
tokenized_essays = {label: [w for w in token_lst if (w not in english_stopwords and w not in custom_stopwords)] for (label, token_lst) in tokenized_essays.items()}

# Vectorize w/ doc2vec

In [9]:
LabeledSentence1 = gensim.models.doc2vec.TaggedDocument
all_content_train = []
j=0
for essay in tokenized_essays.values():
    all_content_train.append(LabeledSentence1(essay, [j]))
    j+=1
    
print("Number of texts processed: ", j)

Number of texts processed:  1577


In [10]:
#d2v_model = Doc2Vec(all_content_train, vector_size = 100, window = 10, min_count = 500, workers=7, dm = 1,alpha=0.025, min_alpha=0.001)
d2v_model = Doc2Vec(all_content_train)

#d2v_model.train(all_content_train, total_examples=d2v_model.corpus_count, epochs=10, start_alpha=0.002, end_alpha=-0.016)

In [11]:
essay_vectors = d2v_model.docvecs.vectors_docs
vectorized_df = pd.DataFrame(essay_vectors)
index_ref = vectorized_df.index

# Feature scaling through standardization

In [12]:
stdsclr = StandardScaler()
standardized_df = pd.DataFrame(stdsclr.fit_transform(vectorized_df.astype(float)), index=index_ref)

# Principle component analysis

In [13]:
pca = PCA(n_components=3)
reduced_df = pd.DataFrame(pca.fit_transform(standardized_df), index=index_ref)

# Guide for output to visualize effectiveness of vectors

In [14]:
#reduced_df.to_csv('new1.csv', sep='\t', index=False, header=False)
#pd.DataFrame(index_ref).to_csv('index.csv', index=False, header=False)

# Clustering w/ k-means 

In [15]:
num_clusters = 7

km = KMeans(n_clusters=num_clusters, init="k-means++", max_iter=100)

%time km.fit(reduced_df.values)

Wall time: 116 ms


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=7, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

# Essays per cluster / Theme(s) per cluster

In [16]:
output = reduced_df
output['cluster'] = km.labels_
print(output['cluster'].value_counts())
output['essay'] = tokenized_essays.values()
output['title'] = tokenized_essays.keys()

6    571
1    355
3    288
0    209
4     90
5     62
2      2
Name: cluster, dtype: int64


In [17]:
theme_index = {
    'Autobiographical, Paths to Prison': 'parent, damage, abuse, gang, alcohol, drug, neighborhood, hood, youth, pressure, fit, broken',
    'Family': 'family, abandonment, relation, visit, partner, mother, father, sibling, child, wife, husband, abuse, support', 
    'Physical Conditions and Security': 'physical, condition, security, search, censorship, food, cold, hygiene, heat, misfunction, infestation, solitary, strip, search, fear, filth, violence, staff, abuse', 
    'Prison Culture/Community/Society': 'violence, fear, staff, sexual, crime, outcasts, racial, cellmate, gay, LGBTQ, dehumanize, uniform, pecking, order, hierarchy, solid, dirty, skin, chomo',
    'Staff/prison Abuse of IP': 'abuse, sexual, torture, humiliation, racist, assault, antagonism, exacerbation, right, violation, food, hygiene, environment, legal, extraction, search, taunt',
    'Personal/Internal Change/Coping': 'survival, art, reading, writing, peace, faith, prayer, meditation, practice, community, activities, hobbies, cooking, remorse, motivation, education, discipline, coping, adjustment, responsibility, god, redemption, transformation',
    'Judicial Misconduct and Legal Remediation': 'judicial, incompetence, corruption, witness, evidence, excessive, political, jailhouse, lawyer, misconduct, unfair, pretender, plra, plea, grievance',
    'Political and Intellectual Labor among IP': 'activism, resistence, critique, race, class, change, policies, practices, write, organize, strike, solidarity',
    'Prison Industry/Prison as Business': 'labor, slave, condition, safety, health, profit, job, budget, tax, taxpayer, exploitation, corruption, mismanagement, nepotism',
    'Education, Re-entry, Other Programs': 'rehabilitation, entry, education, indifference, college, vocation',
    'Health Care': 'health, care, negligence, hostility, incompetence, indifference, death, injury, treatment, medication',
    'Social Alienation, Indifference, Hostility': 'public, misperception, identity, stigma'}

In [18]:
import os
from os import path
import shutil

src = "./APWA_text/Essays/"
dst = "./clusters_1_0/cluster_"

#files = [i for i in os.listdir(src) if path.isfile(path.join(src, i))]
#for f in files:
#    shutil.copy(path.join(src, f), dst)

In [19]:
from __future__ import print_function
import random

order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

essays_per_cluster = {}
for i in range(num_clusters):
    essays_per_cluster[i] = []
    #print("Cluster %d titles:" % i, end='')
    for title in output[output.cluster == i]['title'].values.tolist():
        #print(' %s,' % title, end='')
        essays_per_cluster[i].append(title)
        
print("***** Random sample essays per cluster *****")
print()

terms_per_essays = {}
themes_per_essays = {}
for i in range(num_clusters):
    ten = []
    if len(essays_per_cluster[i]) < 10:
        ten = essays_per_cluster[i]
    else:
        while len(ten) < 10:
            essay = random.choice(essays_per_cluster[i])
            if essay not in ten:
                ten.append(essay)

    print("** Cluster %d: **" % i)
    print()
    context = [' '.join(tokens) for tokens in list(output[output.cluster == i].essay)]
    
    tfidf_vectorizer = TfidfVectorizer(max_features=50)
    tfidf_vectorizer.fit(context)
    
    #terms_per_essays[i] = [term for term in tfidf_vectorizer.get_feature_names() if term not in english_stopwords + custom_stopwords]
    terms_per_essays[i] = [term for term in tfidf_vectorizer.get_feature_names()]
    
    print("Top terms #{} : {}".format(i, terms_per_essays[i]))
    print()
    
    themes_per_essays[i] = set()
    theme_term_count = {}
    for term in terms_per_essays[i]:
        for key, value in theme_index.items():
            if term in value:
                if key in theme_term_count:
                    theme_term_count[key] += 1
                else:
                    theme_term_count[key] = 1
                themes_per_essays[i].add(key)
    
    themes_per_essays[i] = [theme for theme, count in theme_term_count.items() if count > 1]
    print("Dominant theme(s): {}".format(themes_per_essays[i]))
    print()
    print(sorted(theme_term_count.items(), key=lambda x : x[1])[::-1])
    
    print()
    print("Randomly selected essays:", ten)
    files = [i for i in os.listdir(src) if i in ten and path.isfile(path.join(src, i))]
    for f in files:
        shutil.copy(path.join(src, f), dst + str(i))
    print()
    print()


***** Random sample essays per cluster *****

** Cluster 0: **

Top terms #0 : ['american', 'behavior', 'black', 'california', 'cause', 'child', 'community', 'court', 'crime', 'criminal', 'death', 'drug', 'experience', 'fact', 'family', 'federal', 'force', 'help', 'however', 'human', 'incarceration', 'individual', 'inmate', 'issue', 'justice', 'law', 'learn', 'mean', 'member', 'offender', 'officer', 'order', 'parole', 'police', 'problem', 'process', 'program', 'provide', 'public', 'release', 'right', 'sentence', 'serve', 'social', 'society', 'staff', 'state', 'system', 'think', 'write']

Dominant theme(s): ['Family', 'Prison Culture/Community/Society']

[('Prison Culture/Community/Society', 4), ('Family', 2), ('Political and Intellectual Labor among IP', 1), ('Physical Conditions and Security', 1), ('Staff/prison Abuse of IP', 1), ('Social Alienation, Indifference, Hostility', 1), ('Judicial Misconduct and Legal Remediation', 1), ('Autobiographical, Paths to Prison', 1), ('Health Care'

# Running LDA on each cluster 

In [25]:
from gensim import corpora, models

terms_per_cluster = {}
themes_per_cluster = {}
essays_per_cluster = {}
essays_with_themes = {}

#Apply LDA for each cluster (and for each essay in cluster)
for i in range(num_clusters):
    print("************ Cluster # {} ************".format(i))
    essays_in_cluster = [tokens for tokens in list(output[output.cluster == i].essay)]
    
    # Store titles of all essays in a given cluster.
    essays_per_cluster[i] = list(output[output.cluster == i].title)
    
    dictionary = corpora.Dictionary(essays_in_cluster)
    
    cluster_corpus = [dictionary.doc2bow(essay) for essay in essays_in_cluster]
    
    lda = models.ldamodel.LdaModel(corpus=cluster_corpus, id2word=dictionary, num_topics=1, passes=10)
    
    term_score = {}
    terms_per_cluster[i] = []
    for idx, terms in lda.print_topics(i, 20):
        #terms_per_cluster[i] = terms
        
        #print('Top terms: {}'.format(terms.split('+')))
        #print()
        #print()
        
        for term_with_score in terms.split('+'):
            term = term_with_score.split('*')[1][1:-2]
            score = term_with_score.split('*')[0]
            
            #print("term is {} with score {}".format(term, score))
            
            terms_per_cluster[i].append(term)
            term_score[term] = float(score)
    
    #print("Terms per cluster:", terms_per_cluster)
    #print()
    #print("Terms/Scores:", term_score)
    #print()
    
    themes_per_cluster[i] = {}
    theme_term_score = {}
    for term in terms_per_cluster[i]:
        for theme, defining_terms in theme_index.items():
            if term in defining_terms:
                if theme in theme_term_score:
                    theme_term_score[theme] += term_score[term]
                else:
                    theme_term_score[theme] = term_score[term]
                
    themes_per_cluster[i] = sorted(theme_term_score.items(), key = lambda x : x[1])[::-1]
    print("Cluster themes ranked strongest to weakest: {}".format(themes_per_cluster[i]))
    print()
    
    themes_per_essay = {}
    terms_per_essay = {}
    for title in essays_per_cluster[i]:
        essay = [tokens for tokens in list(output[output.title == title].essay)]
        dictionary = corpora.Dictionary(essay)
        essay_corpus = [dictionary.doc2bow(e) for e in essay]
        lda = models.ldamodel.LdaModel(corpus=essay_corpus, id2word=dictionary, num_topics=1, passes=10)
    
        essay_term_score = {}
        terms_per_essay[title] = []
        for idx, terms in lda.print_topics(i, 20):
            
            for term_with_score in terms.split('+'):
                term = term_with_score.split('*')[1][1:-2]
                score = term_with_score.split('*')[0]

                terms_per_essay[title].append(term)
                essay_term_score[term] = float(score)

        themes_per_essay[title] = {}
        essay_theme_term_score = {}
        for term in terms_per_essay[title]:
            for theme, defining_terms in theme_index.items():
                if term in defining_terms:
                    if theme in essay_term_score:
                        essay_theme_term_score[theme] += essay_term_score[term]
                    else:
                        essay_theme_term_score[theme] = essay_term_score[term]

        themes_per_essay[title] = sorted(essay_theme_term_score.items(), key = lambda x : x[1])[::-1]
        
    print("Total number of essays in this cluster: {}.".format(len(essays_per_cluster[i])))
    print()
    
    for title in essays_per_cluster[i][:10]:
        print("Essay ({}) themes: {}".format(title, sorted(themes_per_essay[title], key = lambda x : x[1])))
        print()
        
    essays_with_themes.update(themes_per_essay)

************ Cluster # 0 ************
Cluster themes ranked strongest to weakest: [('Judicial Misconduct and Legal Remediation', 0.004), ('Staff/prison Abuse of IP', 0.003), ('Prison Culture/Community/Society', 0.003), ('Political and Intellectual Labor among IP', 0.002), ('Personal/Internal Change/Coping', 0.002), ('Family', 0.002), ('Health Care', 0.002)]

Total number of essays in this cluster: 209.

Essay (apw_12341398.txt) themes: [('Social Alienation, Indifference, Hostility', 0.012), ('Health Care', 0.012), ('Education, Re-entry, Other Programs', 0.012), ('Prison Industry/Prison as Business', 0.012), ('Political and Intellectual Labor among IP', 0.012), ('Personal/Internal Change/Coping', 0.012), ('Staff/prison Abuse of IP', 0.012), ('Physical Conditions and Security', 0.012), ('Family', 0.012), ('Judicial Misconduct and Legal Remediation', 0.012), ('Prison Culture/Community/Society', 0.014)]

Essay (apw_12341407.txt) themes: [('Political and Intellectual Labor among IP', 0.004)

Cluster themes ranked strongest to weakest: [('Staff/prison Abuse of IP', 0.004), ('Family', 0.002), ('Political and Intellectual Labor among IP', 0.002)]

Total number of essays in this cluster: 90.

Essay (apw_12341322.txt) themes: [('Political and Intellectual Labor among IP', 0.015)]

Essay (apw_12341370.txt) themes: [('Judicial Misconduct and Legal Remediation', 0.004), ('Personal/Internal Change/Coping', 0.007), ('Autobiographical, Paths to Prison', 0.007)]

Essay (apw_12342290.txt) themes: [('Staff/prison Abuse of IP', 0.002)]

Essay (apw_12342923.txt) themes: [('Staff/prison Abuse of IP', 0.007), ('Personal/Internal Change/Coping', 0.007), ('Education, Re-entry, Other Programs', 0.009)]

Essay (apw_12342934.txt) themes: [('Political and Intellectual Labor among IP', 0.012), ('Family', 0.022)]

Essay (apw_12343370.txt) themes: [('Staff/prison Abuse of IP', 0.006), ('Autobiographical, Paths to Prison', 0.009)]

Essay (apw_12343577.txt) themes: [('Personal/Internal Change/Coping',

# Save model/Load from pickle

In [ ]:
joblib.dump(km,'doc_cluster.pkl')

#km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

# Visualize cluster (2D)

In [ ]:
#import random

#%matplotlib inline
#plt.figure

#cluster_colors = []
#for i in range(num_clusters):
#    r = lambda: random.randint(0,255)
#    cluster_colors.append('#%02X%02X%02X' % (r(),r(),r()))

#color = [i for i in cluster_colors]
#plt.scatter(datapoint[:, 0], datapoint[:, 1])
#centroids = kmeans_model.cluster_centers_
#centroidpoint = pca.transform(centroids)
#plt.scatter(centroidpoint[:, 0], centroidpoint[:, 1], marker="^", s=150, c="#000000")
#plt.show()

In [31]:
joblib.dump('essayThemeDict.joblib', essays_with_themes)
joblib.dump('theme_index.joblib', theme_index)

ValueError: Second argument should be a filename or a file-like object, {'apw_12341398.txt': [('Prison Culture/Community/Society', 0.014), ('Social Alienation, Indifference, Hostility', 0.012), ('Health Care', 0.012), ('Education, Re-entry, Other Programs', 0.012), ('Prison Industry/Prison as Business', 0.012), ('Political and Intellectual Labor among IP', 0.012), ('Personal/Internal Change/Coping', 0.012), ('Staff/prison Abuse of IP', 0.012), ('Physical Conditions and Security', 0.012), ('Family', 0.012), ('Judicial Misconduct and Legal Remediation', 0.012)], 'apw_12341407.txt': [('Prison Culture/Community/Society', 0.008), ('Political and Intellectual Labor among IP', 0.004), ('Judicial Misconduct and Legal Remediation', 0.004)], 'apw_12342097.txt': [('Education, Re-entry, Other Programs', 0.009), ('Personal/Internal Change/Coping', 0.009), ('Prison Culture/Community/Society', 0.004), ('Staff/prison Abuse of IP', 0.004)], 'apw_12342180.txt': [('Staff/prison Abuse of IP', 0.004)], 'ap